In [1]:
import os
import django
import json 
#os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from pprint import pprint



In [2]:
from edly.core.models import *

In [7]:
state_obj=State.objects.all()

In [8]:
state_obj

<QuerySet [<State: New York>, <State: Puerto Rico>, <State: Virgin Islands>, <State: Massachusetts>, <State: Rhode Island>, <State: New Hampshire>, <State: Maine>, <State: Connecticut>, <State: New Jersey>, <State: Pennsylvania>, <State: Delaware>, <State: District of Columbia>, <State: Virginia>, <State: Maryland>, <State: North Carolina>, <State: South Carolina>, <State: Georgia>, <State: Florida>, <State: Alabama>, <State: Tennessee>, '...(remaining elements truncated)...']>

# Serialization

In [31]:
from rest_framework import serializers
from rest_framework.renderers import JSONRenderer

In [32]:
class StateSerializer(serializers.ModelSerializer):
    class Meta:
        model = State
        fields = ['id','name','country','alpha_2_iso_code']

In [33]:
serializer=StateSerializer(state_obj,many=True)

In [34]:
serialized_data=serializer.data[0]

In [35]:
json_data= JSONRenderer().render(serialized_data)


In [36]:
json_data

b'{"id":473,"name":"New York","country":3,"alpha_2_iso_code":"NY"}'

# De-Serialization

In [77]:
import io
from rest_framework.parsers import JSONParser

stream = io.BytesIO(json_data)
data = JSONParser().parse(stream)

In [80]:
class StateSerializer2(serializers.ModelSerializer):
    class Meta:
        model = State
        fields = ['id','name','country','alpha_2_iso_code']
    
    def create(self, validated_data):
        """
        Create and return a new `Snippet` instance, given the validated data.
        """
        return State.objects.create(**validated_data)
     
    
    def update(self, instance, validated_data):
        """
        Update and return an existing `Snippet` instance, given the validated data.
        """
        instance.id = validated_data.get('id', instance.id)
        instance.name = validated_data.get('name', instance.name)
        instance.country = validated_data.get('country', instance.country)
        instance.alpha_2_iso_code = validated_data.get('alpha_2_iso_code', instance.alpha_2_iso_code)
        instance.save()
        return instance


In [81]:
data

{'id': 473, 'name': 'New York', 'country': 3, 'alpha_2_iso_code': 'NY'}

In [82]:
serializer = StateSerializer(data=data)

In [83]:
serializer.is_valid()

True

In [84]:
serializer.validated_data

OrderedDict([('name', 'New York'),
             ('country', <Country: United States>),
             ('alpha_2_iso_code', 'NY')])

In [85]:
if serializer.is_valid():
    serializer.validated_data
    serializer.save()
    

In [86]:
serializer.data

{'id': 535, 'name': 'New York', 'country': 3, 'alpha_2_iso_code': 'NY'}

In [88]:
State.objects.filter(id=535)

<QuerySet [<State: New York>]>

# Queries

.

###### all() , filter() , exclude() , order_by( 'field' / '-field' / '?' ) , reverse() , values() , union(all=False) , union(all=True)

.

In [89]:
from edly.core.models import *

In [104]:
# The all() method returns a QuerySet of all the objects in the database.
State.objects.all()[:5]

<QuerySet [<State: New York>, <State: Puerto Rico>, <State: Virgin Islands>, <State: Massachusetts>, <State: Rhode Island>]>

In [96]:
# Returns a new QuerySet containing objects that match the given lookup parameters.
State.objects.filter(name='New York')

<QuerySet [<State: New York>, <State: New York>, <State: New York>, <State: New York>]>

In [97]:
# __startwith 
State.objects.filter(name__startswith="New")

<QuerySet [<State: New York>, <State: New Hampshire>, <State: New Jersey>, <State: New Mexico>, <State: New York>, <State: New York>, <State: New York>]>

In [93]:
# Returns a new QuerySet containing objects that do not match the given lookup parameters.
State.objects.exclude(name='New York')

<QuerySet [<State: Puerto Rico>, <State: Virgin Islands>, <State: Massachusetts>, <State: Rhode Island>, <State: New Hampshire>, <State: Maine>, <State: Connecticut>, <State: New Jersey>, <State: Pennsylvania>, <State: Delaware>, <State: District of Columbia>, <State: Virginia>, <State: Maryland>, <State: North Carolina>, <State: South Carolina>, <State: Georgia>, <State: Florida>, <State: Alabama>, <State: Tennessee>, <State: Mississippi>, '...(remaining elements truncated)...']>

In [181]:
# values() return the obj keys nad value in form of dictionary whereas values_list(named=False) return tuple
State.objects.order_by("id").reverse()[:5].values()

<QuerySet [{'id': 535, 'slug': 'new-york-39nwy', 'name': 'India', 'country_id': 3, 'alpha_2_iso_code': 'NY'}, {'id': 534, 'slug': 'new-york-45cqn', 'name': 'New York', 'country_id': 3, 'alpha_2_iso_code': 'NY'}, {'id': 533, 'slug': 'new-york-83pbz', 'name': 'New York', 'country_id': 3, 'alpha_2_iso_code': 'NY'}, {'id': 532, 'slug': 'west-virginia', 'name': 'West Virginia', 'country_id': 3, 'alpha_2_iso_code': 'WV'}, {'id': 531, 'slug': 'alaska', 'name': 'Alaska', 'country_id': None, 'alpha_2_iso_code': 'AK'}]>

In [ ]:
# union() is used to combine two queries from diffrent database and return a single queryset with distinct values . Only applies when the number of column of both the query is same . 
# union(all=True) is to combine two queries from diffrent database and return a single queryset inculding duplicate values . 
qs2.union(qs1,all=True)
# intersection : to return the shared values 
qs1.intersection(qs2,qs3,qs4)
#difference : all except the shared values 


.

###### get() , __iexact , first() , last() , latest() , earliest() , exists() 

.

In [121]:
State.objects.get(id=535)

<State: New York>

In [122]:
# A case-insensitive match. 
State.objects.filter(name__iexact="nEw YoRk" , id = 535 )

<QuerySet [<State: New York>]>

In [135]:
# first obj that match query . if the method has no oder_by field by default it is ordered by pk
# State.objects.first()
State.objects.order_by("country").first()

<State: New York>

In [136]:
# Similar to first 
State.objects.order_by("country").last()

<State: Palau>

In [139]:
# returns latest obj of the table of given field
State.objects.latest("name")

<State: Wyoming>

In [141]:
# returns earliest obj of the table of given field
State.objects.earliest("name")

<State: Alabama>

In [145]:
State.objects.filter(name="New York").exists()

True

.

###### create() , get_or_create() , update() , update_or_create() , bulk_create() , bulk_update() , in_bulk() , delete()

.

In [222]:
# create obj and saving in one step 

# using save()
dic= {
    "name":"India", 
    "country":3,
    "alpha_2_iso_code":"IND"
}

serializer = StateSerializer(data=dic)
serializer.is_valid()
if serializer.is_valid():
    serializer.save()
else:
    print("Not Valid")
    
    
# using create()
State.objects.create(name="India",alpha_2_iso_code="IND")

False

In [161]:
# get_or_create() : it will get() the obj and created (can be true or false ) if already present....else will create the obj 
State.objects.get_or_create(id= 535, name= 'New York', country= 3, alpha_2_iso_code= 'NY')


(<State: New York>, False)

In [167]:
# update() : cannot use get() instead of filter as it applies only on a Queryset . 
State.objects.filter(id=535).update(name="India")
State.objects.get(id=535)

<State: India>

In [170]:
# update_or_create() : if exist update else create the obj
data, created = State.objects.update_or_create(id= 535, name= 'India', country= 3, alpha_2_iso_code= 'NY')
created

False

In [174]:
# bulk_create(objs,batch_size=None,ignore_conflicts=False) : insert the provided list in db . T&C applied :)
objs=[
    State(id= 535, name= 'New York', country= 3, alpha_2_iso_code= 'NY'),
    State(id= 536, name= 'New York', country= 3, alpha_2_iso_code= 'NY'),
    State(id= 537, name= 'New York', country= 3, alpha_2_iso_code= 'NY'),
]
State.objects.bulk_create(objs)

# bulk_update() : update the provided list in db 
State.objects.bulk_update(objs)

ValueError: Cannot assign "3": "State.country" must be a "Country" instance.

In [176]:
# in_bulk : retrive objs in bulk 
State.objects.in_bulk([535,473])

{473: <State: New York>, 535: <State: India>}

.

###### Q objects

In [197]:
# Q objs are the objs used to encapsulate a collection of keyword arguments . These keywords as specified as in "Field Lookups"


###### OR / AND / NOT 

.

In [190]:
# OR : Query would return even if ant one Query is True
State.objects.filter(id=535)|State.objects.filter(name="New York")
State.objects.filter(Q(id=535)|Q(name="New York"))

<QuerySet [<State: New York>, <State: New York>, <State: New York>, <State: India>]>

In [186]:
# AND : Query would return only when both Query is True
State.objects.filter(id=535)&State.objects.filter(name="India")
State.objects.filter(id=535,name="India")
State.objects.filter(Q(id=535)&Q(name="India"))

<QuerySet [<State: India>]>

In [196]:
# NOT : anti 
State.objects.filter(~Q(name="New York"))


<QuerySet [<State: Puerto Rico>, <State: Virgin Islands>, <State: Massachusetts>, <State: Rhode Island>, <State: New Hampshire>, <State: Maine>, <State: Connecticut>, <State: New Jersey>, <State: Pennsylvania>, <State: Delaware>, <State: District of Columbia>, <State: Virginia>, <State: Maryland>, <State: North Carolina>, <State: South Carolina>, <State: Georgia>, <State: Florida>, <State: Alabama>, <State: Tennessee>, <State: Mississippi>, '...(remaining elements truncated)...']>

.

 ###### F expressions

.

In [ ]:
# we gernally compare the value of a model field with a constant  but F expressions are used to compare the value of a model field with another field on the same model .

from django.db.models import F

# Entry.objects.filter(number_of_comments__gt=F('number_of_pingbacks'))
# here the number_of_comments (value of model Field )that are greater the number_of_pingbacks (value of model Field ) are getting filtered .



.

###### annotate() , aggregate(Avg/Sum/Min/Max/Count)

.

In [203]:
State.objects.aggregate(Avg("country"))
#State.objects.aggregate(Sum("country"))
#State.objects.aggregate(Min("country"))
#State.objects.aggregate(Max("country"))
#State.objects.aggregate(Count("country"))





{'country__avg': 3.0}

In [215]:
# annotate is used to dynamically generate additional data for each object when the data is being extracted.
annotated_data=State.objects.annotate(new_id=F("id")*10)

In [221]:
annotated_data.values("new_id")

<QuerySet [{'new_id': 4730}, {'new_id': 4740}, {'new_id': 4750}, {'new_id': 4760}, {'new_id': 4770}, {'new_id': 4780}, {'new_id': 4790}, {'new_id': 4810}, {'new_id': 4820}, {'new_id': 4830}, {'new_id': 4840}, {'new_id': 4850}, {'new_id': 4860}, {'new_id': 4870}, {'new_id': 4890}, {'new_id': 4900}, {'new_id': 4910}, {'new_id': 4920}, {'new_id': 4930}, {'new_id': 4940}, '...(remaining elements truncated)...']>